In [2]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
import codecs

# Introduction

In this notebook a step-by-step symbolic solution to obtain the kernel $K(x|u)$ for the problem

$f(x)(1+W(x) + W(x+1)) - W(x)f(x-1) - W(x+1)f(x+1) = g(x)$

when  the weights are given by a plateau at $x= 0$ with 

$W(x) = \left\{\begin{matrix}
\lambda\,, & x< 0\\
\mu, & x \geq 0\\
\end{matrix}\right.$

The kernel is shown to be $K(x|u) = a(x)a(u)e^{-|t(x)-t(u)|}$, where $t(x)$ is given implicitly by $\Delta t(x) = t(x)-t(x-1)$.

We note, however, that Sympy is not powerful enough to simplify the expressions obtained here. To obtain more concise solutions, we recommend using a specialized algebra computing system such as Wolfram's Mathematica.

# General solution for the plateau at $x_0 = 0$

We begin by defining the weight function W(x)

In [3]:
x= sp.symbols('x')
w0= sp.symbols('w0')
λ= sp.symbols('λ')
n = sp.symbols('n')
μ = sp.symbols('μ')

W = sp.Piecewise(
    (λ, x <0),
    (μ, x>=0),
)

display(Latex(r"$W(x) = $ " + sp.latex(W, mode='inline')))

<IPython.core.display.Latex object>

# Explicit plateau domain transform by symbolic computing

The $γ(λ)$ is the default value for the $B(x)$ coefficients in homogeneous regions, computed by continued fractions

$B(x) = 1 + W(x) + W(x+1)  - W(x+1)^2/B(x+1)$

$B(x) = 1 + λ + λ  - λ^2/B(x+1)$

$B(x) = 1 + 2λ - \frac{λ^2}{ 1 + 2λ - \frac{λ^2}{ 1 + 2λ \ldots}} = γ(λ)$

# Computing B

If, however, $B(x)$ is not $\textbf{all}$ equal to $\gamma(\lambda)$, then we can still compute $B(x-k)$ for arbitrary choice of $k$ using matrix multiplication. The idea is as follows:

Suppose $B(x) = \frac{p}{q}$ for some fraction $p/q$, and that the weights are all homogeneous for $x$ in $x+1$ to $-\infty$.

$B(x-1) = 1 + 2\lambda - \frac{\lambda^2}{\frac{p}{q}}$

is simplified to

$B(x-1) = \frac{νp- λ^2q}{p}$, which means each step $x \mapsto x-1$ is written in matrix form as 

$      \begin{bmatrix}
\nu  & -\lambda^2\\ 
1 & 0
\end{bmatrix}
\begin{bmatrix}
p\\ 
r
\end{bmatrix} \mapsto \begin{bmatrix}
\nu p - \lambda^2 r\\ 
p
\end{bmatrix}\,,$

Therefore $B(x-k)$ for $k>=0$ can be written in terms of $B(x) = \frac{p}{q}$ with

$  \frac{\text{numerator}(B(x-k))}{\text{denominator}(B(x-k))}  =   \begin{bmatrix}
\nu  & -\lambda^2\\ 
1 & 0
\end{bmatrix}^k
\begin{bmatrix}
p\\ 
r
\end{bmatrix} $


We begin my writing down the variables

In [4]:
γ =  (2 * λ + sp.sqrt(4 * λ + 1) + 1)/2
ν = 1 + 2*λ
Bstep_mat = sp.Matrix([[ν, -λ*λ], [1, 0]])

display(Latex(r"$γ = $ " + sp.latex(γ, mode='inline')))
display(Latex(r"$ν = $ " + sp.latex(ν, mode='inline')))
display(Bstep_mat)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Matrix([
[2*λ + 1, -λ**2],
[      1,     0]])

## Diagonalizing B(x-k)

In [11]:
k = sp.symbols('k')
bstart = sp.symbols('bs')
P, D = Bstep_mat.diagonalize()
Pinv = P.inv()

Bsteps_mat = (P * (D**k) * P.inv())*sp.Matrix([[bstart], [1]]) #bstart is the starting value of b(x) to compute b(x-k)

Bsteps = Bsteps_mat[0]/Bsteps_mat[1]

#the starting fracion can be chosen conveniently, we simply make q=1 to simplify computation
display(Latex(r"We then have the expression, starting from $B(-1)$ for decreasing indices"))

display(Latex(r" $B(-k-1) =$ " + sp.latex(Bsteps.subs(bstart, sp.symbols('B(-1)')), mode='inline')))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## B as a pure function

We write down the function $B(x)$ as a piecewise function. Notice that it is constant equal to $\gamma(\mu)$ for $x\geq0$. Then

\begin{matrix}
B(0) & =  \gamma(\mu)\\
B(-1) & = 1 + \lambda + \mu - \mu^2/B(0)\\
B(-2) & = 1 + 2\lambda - \lambda^2/B(0)
\end{matrix}

In [18]:
Bx0 = γ.subs(λ,μ)
Bxm1 =  1 + W.subs(x,-1)+ W.subs(x,0) - W.subs(x,0)**2/Bx0


B = sp.Piecewise(
    (Bx0 , (x>=0)), #at x>=0
    (Bsteps.replace(k,-x-1).replace(bstart, Bxm1), x<=-1)
)

display(Latex(r"We obtain the large expression"))

display(Latex(r" $B(x) =$ " + sp.latex(B, mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Diagonalizing B2

Similarly we can solve for $B_2(x) = 1 + W(x) + W(x+1) - W(x)^2/B_2(x-1)$. The key difference is that we are computing a forward step $B_2(x+k)$ as a function of $B(x)$. The steps are taken at the right side of the plateau, where the weights are all $\mu$.

In [19]:
bstart2 = sp.symbols('bs2')
B2steps = Bsteps.replace(bstart,bstart2)

display(Latex(r"We then have the expression, starting from $B_2(1)$ for increasing indices"))
display(Latex(r" $B_2(k+1) =$ " + sp.latex(B2steps.subs(bstart2,sp.symbols(r'B_{2}(1)')).subs(λ,μ), mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## B2 as a pure function

We write down the function $B_2(x)$ as a piecewise function. Notice that it is constant equal to $\gamma(\lambda)$ for $x\leq-2$. Then

\begin{matrix}
B_2(-2) & =  \gamma(\lambda)\\ 
B_2(-1) & =  1 + \lambda + \mu - \frac{\lambda^2}{B_2(-2)}\\
B_2(0) & = 1 + 2\mu - \frac{\mu^2}{B_2(-1)}
\\
B_2(1) & = 1 + 2\mu - \frac{\mu^2}{B_2(0)}
\end{matrix}


In [33]:
B2xm2 = γ
B2xm1 = 1 + W.subs(x,-1)+ W.subs(x,0) - W.subs(x,-1)**2/B2xm2
B2x0 =  1 + W.subs(x,0)+ W.subs(x,1) - W.subs(x,0)**2/B2xm1


B2 = sp.Piecewise(
    (B2xm2, x <= -2),
    (B2xm1, (x < 0)&(x>-2)), #at x==-1
    (B2steps.replace(k,x).replace(bstart2, B2x0).replace(λ,μ), x>=0)
)

display(Latex(r"So"))

display(Latex(r" $B_2(x) =$ " + sp.latex(B2, mode='inline')))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Squared normalization coefficients $a^2$ and Domain transform dt

We use that 

$ a(x)^2 = \frac{1}{B(x) - \frac{W(x)^2}{B_2(x-1)}}\,.$

and

$ (t(x)-t(x-1)) = -\log \left( \frac{-1 + \sqrt{4 a(x-1)^2 a(x)^2W(x)^2+1}}{2 a(x-1)a(x)W(x)}\right)$

In [29]:
asqr = 1/(B- W*W/(B2.replace(x,x-1)))

# Domain transform

The domain transform is then given by

In [62]:
dt = sp.log((2*sp.sqrt(asqr.replace(x,x-1)*asqr)*W)/(-1 + sp.sqrt(4*asqr.replace(x,x-1)*asqr*W*W+1)))
display(dt)

log(2*sqrt(1/((-Piecewise((λ**2, x < 0), (μ**2, True))*Piecewise((1/(λ + sqrt(4*λ + 1)/2 + 1/2), x <= -1), (1/(-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + λ + μ + 1), x < 1), (((-(μ - sqrt(4*μ + 1)/2 + 1/2)**(x - 1)/sqrt(4*μ + 1) + (μ + sqrt(4*μ + 1)/2 + 1/2)**(x - 1)/sqrt(4*μ + 1))*(-μ**2/(-μ**2/(μ + sqrt(4*μ + 1)/2 + 1/2) + 2*μ + 1) + 2*μ + 1) + (-2*μ + sqrt(4*μ + 1) - 1)*(μ + sqrt(4*μ + 1)/2 + 1/2)**(x - 1)/(2*sqrt(4*μ + 1)) + (μ - sqrt(4*μ + 1)/2 + 1/2)**(x - 1)*(2*μ + sqrt(4*μ + 1) + 1)/(2*sqrt(4*μ + 1)))/((-(μ - sqrt(4*μ + 1)/2 + 1/2)*(μ - sqrt(4*μ + 1)/2 + 1/2)**(x - 1)/sqrt(4*μ + 1) + (μ + sqrt(4*μ + 1)/2 + 1/2)*(μ + sqrt(4*μ + 1)/2 + 1/2)**(x - 1)/sqrt(4*μ + 1))*(-μ**2/(-μ**2/(μ + sqrt(4*μ + 1)/2 + 1/2) + 2*μ + 1) + 2*μ + 1) + (-2*μ + sqrt(4*μ + 1) - 1)*(μ + sqrt(4*μ + 1)/2 + 1/2)*(μ + sqrt(4*μ + 1)/2 + 1/2)**(x - 1)/(2*sqrt(4*μ + 1)) + (μ - sqrt(4*μ + 1)/2 + 1/2)*(μ - sqrt(4*μ + 1)/2 + 1/2)**(x - 1)*(2*μ + sqrt(4*μ + 1) + 1)/(2*sqrt(4*μ + 1))), True)) + Piecewise((μ + sqrt(4*μ + 1)/2

# Domain Transform at $x_0$

In the expression

$ (t(0)-t(-1)) = -\log \left( \frac{-1 + \sqrt{4 a(-1)^2 a(0)^2W(0)^2+1}}{2 a(-1)a(0)W(0)}\right)$

We then get the expression at the origin

In [103]:
dt.subs(x,0)

log(2*μ*sqrt(1/((-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + ((-(λ - sqrt(4*λ + 1)/2 + 1/2)/sqrt(4*λ + 1) + (λ + sqrt(4*λ + 1)/2 + 1/2)/sqrt(4*λ + 1))*(λ - μ**2/(μ + sqrt(4*μ + 1)/2 + 1/2) + μ + 1) + (-2*λ + sqrt(4*λ + 1) - 1)*(λ + sqrt(4*λ + 1)/2 + 1/2)/(2*sqrt(4*λ + 1)) + (λ - sqrt(4*λ + 1)/2 + 1/2)*(2*λ + sqrt(4*λ + 1) + 1)/(2*sqrt(4*λ + 1)))/((-2*λ + sqrt(4*λ + 1) - 1)/(2*sqrt(4*λ + 1)) + (2*λ + sqrt(4*λ + 1) + 1)/(2*sqrt(4*λ + 1))))*(-μ**2/(-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + λ + μ + 1) + μ + sqrt(4*μ + 1)/2 + 1/2)))/(sqrt(4*μ**2/((-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + ((-(λ - sqrt(4*λ + 1)/2 + 1/2)/sqrt(4*λ + 1) + (λ + sqrt(4*λ + 1)/2 + 1/2)/sqrt(4*λ + 1))*(λ - μ**2/(μ + sqrt(4*μ + 1)/2 + 1/2) + μ + 1) + (-2*λ + sqrt(4*λ + 1) - 1)*(λ + sqrt(4*λ + 1)/2 + 1/2)/(2*sqrt(4*λ + 1)) + (λ - sqrt(4*λ + 1)/2 + 1/2)*(2*λ + sqrt(4*λ + 1) + 1)/(2*sqrt(4*λ + 1)))/((-2*λ + sqrt(4*λ + 1) - 1)/(2*sqrt(4*λ + 1)) + (2*λ + sqrt(4*λ + 1) + 1)/(2*sqrt(4*λ + 1))))*(-μ**2/(-λ**2/(λ + sqrt(4*λ + 1)/2 + 1/2) + λ + 

At this point, we recommend using a more powerful software, such as Wolfram's Mathematica to perform a simplification on the expression above